In [1]:
# 레시피
import pandas as pd

food_data = pd.read_csv('../data/전처리후/recipe_list_열량.csv')
food_data

,레시피구분,레시피명,열량
0,볶음류,건새우해바라기씨볶음,열량 : 100.91 kcal
1,밥류,해물볶음덮밥,열량 : 662.26 kcal
2,김치류,알타리김치,열량 : 20.5 kcal
3,김치류,포기김치,열량 : 30.4 kcal
4,김치류,깍두기,열량 : 19 kcal
...,...,...,...
5505,볶음류,무나물볶음,열량 : 21.47 kcal
5506,무침류,고춧잎나물,열량 : 35.88 kcal
5507,무침류,골뱅이무침,열량 : 78.61 kcal
5508,튀김류,수제돈가스,열량 : 224.25 kcal


In [3]:
recipe_data= pd.read_csv('../data/전처리전/recipe_list.csv')
recipe = recipe_data.sort_values(by='레시피명')
recipe = recipe.groupby(['레시피구분', '레시피명']).apply(
    lambda x: x[['식재료명', '식재료양']].reset_index(drop=True)
)
recipe

식재료명  식재료양
레시피구분 레시피명                            
구이류   가자미구이    0             가자미   80g
               1       후춧가루(검은색)  0.1g
               2         소금(천일염)    1g
               3        콩기름(대두유)    3g
               4             밀가루   10g
...                          ...   ...
튀김류   흑임자누룽지치킨 8              후추    1g
               9        분말(겨자분말)    2g
               10          흑임자가루    3g
               11  닭고기(가슴살, 안심살)   30g
               12       닭고기(다리살)   40g

[41579 rows x 2 columns]

In [4]:
food_data.sort_values(by='레시피구분').reset_index(drop=True)

,레시피구분,레시피명,열량
0,구이류,김구이,열량 : 6.96 kcal
1,구이류,돼지LA갈비구이,열량 : 182.48 kcal
2,구이류,파인애플함박스테이크,열량 : 179.73 kcal
3,구이류,피자군만두,열량 : 292.35 kcal
4,구이류,가지구이,열량 : 47.85 kcal
...,...,...,...
5505,튀김류,장어튀김&amp;칠리소스,열량 : 330.41 kcal
5506,튀김류,느타리버섯탕수육,열량 : 139.6 kcal
5507,튀김류,김부각,열량 : 54.96 kcal
5508,튀김류,고구마빠스,열량 : 135.96 kcal


In [5]:
# 빵류 기타류 소스류 제거
food_data = food_data.drop(food_data[food_data['레시피구분'] == '소스류'].index)
food_data = food_data.drop(food_data[food_data['레시피구분'] == '빵류'].index)
food_data = food_data.drop(food_data[food_data['레시피구분'] == '기타'].index)


In [6]:
food_data = food_data.drop_duplicates(subset=['레시피명']).reset_index(drop=True)

In [7]:
food_data.sort_values(by='레시피구분').reset_index(drop=True)
food_data['열량'] = food_data['열량'].str.replace('kcal', '').str.replace('열량 : ', '').str.strip().astype(float)
food_data.to_csv('../data/전처리후/reci_data_float.csv')

In [8]:
# 레시피 구분별로 식단설계
used_reci_name=pd.DataFrame(food_data['레시피구분'].drop_duplicates()).reset_index(drop=True)
used_reci_name

,레시피구분
0,볶음류
1,밥류
2,김치류
3,무침류
4,부침류
5,면류
6,조림류
7,국류
8,구이류
9,찜류


In [9]:
# 조건에 맞는 데이터 필터링

filtered_food_data = food_data[food_data['레시피구분'].isin(['볶음류', '구이류', '조림류', '찜류', '부침류', '튀김류'])]

# 모든 수치형 열의 평균 계산
mean_values = filtered_food_data['열량'].mean()

print(mean_values)

152.7033827014218


In [10]:

select_1 = food_data.loc[food_data['레시피구분'].isin(['밥류', '면류', '죽류'])]
select_2 = food_data.loc[food_data['레시피구분'].isin(['볶음류', '구이류', '조림류','찜류','부침류','튀김류'])]
select_3 = food_data.loc[food_data['레시피구분'].isin(['무침류', '김치류'])]
select_4 = food_data.loc[food_data['레시피구분'].isin(['국류'])]
select_5 = food_data.loc[food_data['레시피구분'].isin(['보조식'])]
def diets_recommend():
    """
    사용자 입력을 받아 BMI와 BMR을 계산하고, 목표 칼로리를 기준으로 음식 추천을 제공합니다.
    부족한 경우 보조식을 추가로 추천하며, 최종 결과는 데이터프레임으로 출력합니다.
    """
    # 사용자 입력 받기
    age = int(input('나이를 입력하세요: '))
    gender = input("성별을 입력하세요 (남자/여자): ")
    height = float(input("신장을 입력하세요 (cm): "))
    weight = float(input("체중을 입력하세요 (kg): "))
    activity_factor = float(input("활동량 계수를 입력하세요:\n"
                                  "활동없음=1.0, 가벼운 움직임=1.15, 숨이찰정도로움직임=1.26, 숨이차고 땀이날정도로 움직임=1.46:\n"))

    # BMI 계산
    BMI = weight / ((height * 0.01) ** 2)

    # BMR 계산
    if gender == '남자':
        BMR = 662 - 9.53 * age + (15.91 * weight + 5.396 * height) * activity_factor
    else:
        BMR = 354 - 6.91 * age + (9.36 * weight + 7.26 * height) * activity_factor

    # BMI 상태 및 권장 칼로리 계산
    if gender == '남자':
        if BMI < 13:
            status = "저체중"
            recommended_calories = (BMR + 500) * 0.33
        elif 13 <= BMI < 25:
            status = "정상"
            recommended_calories = BMR * 0.33
        elif 25 <= BMI < 30:
            status = "경도 비만"
            recommended_calories = BMR * 0.8 * 0.33
        else:
            status = "고도 비만"
            recommended_calories = (BMR - 500 )* 0.33
    else:
        if BMI < 22:
            status = "저체중"
            recommended_calories = (BMR + 500) * 0.33
        elif 22 <= BMI < 34:
            status = "정상"
            recommended_calories = BMR * 0.33
        elif 34 <= BMI < 40:
            status = "경도 비만"
            recommended_calories = BMR * 0.8 * 0.33
        else:
            status = "고도 비만"
            recommended_calories = (BMR - 500) * 0.33

    print(f"\n체질량지수(BMI): {BMI:.2f} ({status})")
    print(f"계산된 일일권장칼로리: {BMR:.2f} kcal")
    print(f"추천하는 한 끼의 칼로리: {recommended_calories:.2f} kcal")

    # 음식 추천 로직
    remaining_calories = recommended_calories
    recommendations = []

    # 1. select_1 추천
    food = select_1.sample(1)
    recommendations.append(food)
    remaining_calories -= food['열량'].sum()

    # 2. select_3 추천
    food = select_3.sample(1)
    recommendations.append(food)
    remaining_calories -= food['열량'].sum()

    # 3. select_2에서 1개 추천
    food = select_2.sample(1)
    recommendations.append(food)
    remaining_calories -= food['열량'].sum()

    # 4. select_4 추천
    food = select_4.sample(1)
    recommendations.append(food)
    remaining_calories -= food['열량'].sum()
    
    # 5. 열량이 부족하면 select_2에서 한번 더 추천
    if remaining_calories > 0:
        food = select_2[select_2['열량'] >= 150].sample(1)  # 열량이 150 이상인 데이터만 샘플링
        if food['열량'].sum() <= remaining_calories*1.10:
            recommendations.append(food)
            remaining_calories -= food['열량'].sum()

    # 5. 열량이 부족하면 select_5에서 추가
    if remaining_calories > 0:
        food = select_5.sample(1)
        if food['열량'].sum() <= remaining_calories:
            recommendations.append(food)
            remaining_calories -= food['열량'].sum()

    # 최종 추천 음식 합치기
    final_recommendations = pd.concat(recommendations)
    total_recommended_calories = final_recommendations['열량'].sum()

    # 결과 데이터프레임 생성
    result_df = final_recommendations[['레시피명', '열량']].copy()
    result_df.loc['총합'] = ['총열량', total_recommended_calories]

    # 출력
    print("\n추천 식단과 식단 레시피:")
    
    detailed_recipes = {}
    for recipe_name in final_recommendations['레시피명']:
        recipe_ingredients = recipe_data[recipe_data['레시피명'] == recipe_name][['식재료명', '식재료양']]
        if not recipe_ingredients.empty:
            detailed_recipes[recipe_name] = recipe_ingredients
        else:
            print(f"레시피 '{recipe_name}'에 대한 정보가 recipe_data에 없습니다.")

    # 레시피별 그룹화된 결과 출력
    grouped_recipes = recipe_data[recipe_data['레시피명'].isin(final_recommendations['레시피명'])]
    grouped_recipes = grouped_recipes[['레시피명', '식재료명', '식재료양']].reset_index(drop=True)
    grouped_recipes = grouped_recipes.groupby(['레시피명']).apply(
        lambda x: x[['식재료명', '식재료양']].reset_index(drop=True))
    return display(result_df), display(grouped_recipes)
   


In [13]:
diets_recommend()

나이를 입력하세요: 28
성별을 입력하세요 (남자/여자): 남자
신장을 입력하세요 (cm): 177
체중을 입력하세요 (kg): 90
활동량 계수를 입력하세요:
활동없음=1.0, 가벼운 움직임=1.15, 숨이찰정도로움직임=1.26, 숨이차고 땀이날정도로 움직임=1.46:
1.15

체질량지수(BMI): 28.73 (경도 비만)
계산된 일일권장칼로리: 3140.20 kcal
추천하는 한 끼의 칼로리: 829.01 kcal

추천 식단과 식단 레시피:


,레시피명,열량
3758,수제비감자탕,79.38
672,사과요거트샐러드,62.79
3537,콩나물버섯찜,44.76
2410,소고기스프,183.90
1936,베이컨떡볶이,189.52
2686,과카몰리바게트샌드위치,124.09
총합,총열량,684.44


식재료명  식재료양
레시피명                         
과카몰리바게트샌드위치 0      아보카도   40g
            1       토마토   20g
            2       바게트   15g
            3        양파   10g
            4        레몬    2g
...                 ...   ...
콩나물버섯찜      5  간장(양조간장)    1g
            6       참기름    1g
            7      고춧가루  0.5g
            8  전분(감자전분)  0.5g
            9   설탕(백설탕)  0.5g

[66 rows x 2 columns]

(None, None)

In [14]:
# 학습한 모델로 한국인 체중 예측
import pandas as pd
pd.set_option('display.max_rows', None) 
pred_data= pd.read_csv('../data/전처리후/korean_data_pred.csv')
pred_data = pred_data[(pred_data['일일 칼로리 잉여/적자'] <= 1000) | (pred_data['일일 칼로리 잉여/적자'] >= -1000)]
pred_data['체중변화예측'] = pred_data['체중변화예측'].astype(str).str.strip('[]')
pred_data['체중변화예측'] = pred_data['체중변화예측'].astype(float)

In [15]:
pred_data

,나이,성별,신장,체중,활동량계수,BMR,에너지 섭취량(Kcal),일일 칼로리 잉여/적자,체중변화예측
0,70.0,2.0,151.2,41.8,1.00,1359.260000,922.289958,-436.970042,-3.644551
1,42.0,1.0,172.5,67.7,1.00,2269.657000,2243.391641,-26.265359,3.673457
2,38.0,2.0,160.9,43.4,1.00,1665.778000,1649.091552,-16.686448,1.388574
3,34.0,2.0,162.8,68.4,1.00,1941.212000,1501.215719,-439.996281,4.997007
4,46.0,1.0,165.3,62.0,1.15,2383.755620,1874.773750,-508.981870,0.880024
5,43.0,1.0,170.7,77.0,1.00,2398.377200,2127.364836,-271.012364,3.476100
6,51.0,2.0,149.6,54.2,1.00,1594.998000,1279.208569,-315.789431,0.232491
7,34.0,2.0,172.2,55.9,1.15,2158.465400,1496.670382,-661.795018,2.074933
8,33.0,2.0,161.9,55.9,1.15,2079.380700,1584.377199,-495.003501,4.167687
9,32.0,1.0,180.1,75.0,1.00,2522.109600,2234.162957,-287.946643,1.356692
